In [1]:
import json

import pandas as pd
from tqdm import tqdm
import tweepy
from dotenv import load_dotenv
import os

from utils import divide_chunks

load_dotenv()  # take environment variables from .env.

True

In [2]:
client = tweepy.Client(bearer_token=os.environ['BEARER'],
                       wait_on_rate_limit=True)

In [3]:
DATA_PATH = 'data'
users_path = os.path.join(DATA_PATH, 'followers_2016.jsonl')
with open(users_path, encoding='utf8') as f:
    users = pd.concat(
        pd.DataFrame([dict(follower, **{'follows':pollster}) for follower in followers])
        for l in f for pollster, followers in json.loads(l).items()
    )
users.head()

,name,description,profile_image_url,id,username,follows,withheld
0,Beautiful flower,"Like fitness do yoga, Play golf for fun not pr...",https://pbs.twimg.com/profile_images/160820000...,1596715669684363264,Alyssa12348597,120664539,NaN
1,Elisa Fenlon,,https://pbs.twimg.com/profile_images/159954889...,1599548681694466049,ElisaFenlon,120664539,NaN
2,Alecia Delbene,🙊Trampoline🧡Artist,https://pbs.twimg.com/profile_images/155788042...,1557880267653533696,AleciaDelbene,120664539,NaN
3,Charleigh Pottier,,https://pbs.twimg.com/profile_images/155937871...,1559378560384507904,CharleighPotti1,120664539,NaN
4,Catherine Seagroves,,https://pbs.twimg.com/profile_images/158861629...,1588616089159843844,CatherineSeagr8,120664539,NaN


In [4]:
# Replace User IDs
ids = users.id.tolist()
ids[:3]

['1596715669684363264', '1599548681694466049', '1557880267653533696']

In [5]:
len(ids)

44618

In [9]:
with open(os.path.join(DATA_PATH, 'followers_2016_rehydrated.jsonl'), 'w+', encoding='utf8') as f:
    for chunk in tqdm(divide_chunks(ids, 100), desc='batch', total = len(ids)//100+bool(len(ids)%100)):
        user_fields = ['created_at','description','entities','id','location','name','pinned_tweet_id','profile_image_url','protected','public_metrics','url','username','verified','withheld']
        returned = client.get_users(ids=chunk, user_fields=user_fields)
        for user in returned.data:
            f.write(json.dumps(user.data)+'\n')

batch: 100%|██████████| 447/447 [16:00<00:00,  2.15s/it]   
